In [7]:
import os
from dotenv import load_dotenv


from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_openai import OpenAIEmbeddings

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

load_dotenv()

True

In [8]:
# Settings
vector_store_address = os.environ.get("YOUR_AZURE_SEARCH_ENDPOINT")
vector_store_password = os.environ.get("YOUR_AZURE_SEARCH_ADMIN_KEY")
openai_api_key = os.environ.get("OPENAI_API_KEY")

In [9]:
# Option 1: Use OpenAIEmbeddings with OpenAI account for  embeddings 
model: str = "text-embedding-ada-002"

embeddings: OpenAIEmbeddings = OpenAIEmbeddings(
    openai_api_key=openai_api_key, model=model
)

In [10]:
# vector store instances
index_name: str = "expotech25_v1"

vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

In [11]:
# Insert text and embeddings into vector store
sources = [
    "./retriever_documents/dsinergia.pdf",
    "./retriever_documents/camara_comercio.pdf",
    "./retriever_documents/informacion_general.pdf"
]
def load_pdf(sources):
    docs = []

    for source in sources:
        loaders = [PyPDFLoader(source)]
        for loader in loaders:
            docs.extend(loader.load())

    return docs

documents = load_pdf(sources)

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

vector_store.add_documents(documents=docs)


['ZDdiYTU2ZjMtOGQxOC00NjQzLTliMGMtMmRiMzE3NzZjMjMx',
 'OTU1MTQwZGQtNjM1YS00ZWEwLTk3OGItZDkxY2VlZjBkYWY0',
 'MjEzNWM3NDAtOWFhZC00ODMzLTgyYTktY2YzOTZlNDM5Zjli',
 'NTg4ZTUxOWYtMjg3Ny00YzU3LWFhOTctYTY4MjRmMGI4YzM0',
 'NWI3ZTQ0MzItNDJhOC00NjdkLWEwYTAtNmU2ODFlNWJiNDNl',
 'NTNjOTI0ZDUtZDFhZC00MjA4LTk2MWEtNzc3MmU1Mzk5NWJh']

In [12]:
# Perform a vector similarity search
docs = vector_store.similarity_search(
    query="que hace Dsinergia?",
    k=3,
    search_type="similarity",
)
print(docs[0].page_content)

Sobre Dsinergia: 
Dsinergia Corp. es una empresa de consultoría con sede en Ciudad de Panamá, Panamá, que se 
especializa en integración de sistemas, diseño, desarrollo, innovación, calidad y seguridad de 
datos. Su misión es colaborar estrechamente con sus clientes para proporcionar sinergia en cada 
oportunidad, maximizando los resultados con el menor esfuerzo posible. El equipo de Dsinergia 
está compuesto por profesionales de diversas disciplinas, como negocios, ingeniería, tecnología y 
programación, con experiencia en países como Panamá, Puerto Rico, México, Venezuela, 
Colombia, Estados Unidos, España y Costa Rica. Desde su fundación en 2012, la empresa se ha 
enfocado en aprovechar la experiencia de su equipo y en crear sinergia en cada una de sus 
iniciativas. 
 
Dsinergia y la Inteligencia Artificial: 
Dsinergia Corp. ha establecido una alianza estratégica con HM Analytical LLC, establecida en 
Estados Unidos en el Estado de la Florida, para crear Visn AI, una empresa que bus

In [13]:
# Perform a vector similarity search with relevance scores

docs_and_scores = vector_store.similarity_search_with_relevance_scores(
    query="cuales son las ventajas de participar de la expotech?",
    k=4,
    score_threshold=0.80,
)
from pprint import pprint

pprint(docs_and_scores)

[(Document(metadata={'id': 'NTg4ZTUxOWYtMjg3Ny00YzU3LWFhOTctYTY4MjRmMGI4YzM0', 'source': './retriever_documents/informacion_general.pdf', 'page': 1}, page_content='5. Actualización en tendencias tecnológicas: Puedes conocer las innovaciones y \ndesarrollos recientes en el sector, manteniéndote al día con los avances \ntecnológicos y aplicando ese conocimiento en tu negocio. \n6. Generación de oportunidades de negocio: La EXPO TECH está diseñada para \nayudar a las empresas a encontrar nuevos socios y clientes. Las oportunidades de \ncolaboración y las ventas pueden surgir durante o después del evento. \n \nPlanes de Exhibición Disponibles: \n1. Plan A: \no Inversión: $1,200.00 + ITBMS \no Incluye un espacio físico de 3 metros x 2 metros, salida eléctrica hasta 500 \nwatts, seguridad general, aire acondicionado, iluminación general y aseo \ngeneral. \n2. Plan B: \no Inversión: $1,500.00 + ITBMS \no Incluye un stand de 3 metros x 2 metros con paredes blancas de melamina, \nuna mesa con d

In [14]:
# Perform a hybrid search using the search_type parameter
docs = vector_store.similarity_search(
    query="cuales son las ventajas de participar de la expotech?",
    k=3,
    search_type="hybrid",
)
print(docs[0].page_content)

EXPOTECH VI Edición, Panamá 2025 
 
Descripción General: 
EXPO TECH 2025 es la feria tecnológica más destacada del año en Panamá, organizada por 
la Cámara de Comercio, Industrias y Agricultura de Panamá (CCIAP). Su Sexta Edición se 
llevará a cabo del 25 al 27 de marzo de 2025 en el Panama Convention Center, ubicado en 
Amador, Ciudad de Panamá. El horario de la expo será de 11:00AM a 7:00PM.  El website 
oficial de la ExpoTech es https://www.panacamara.com/expotech/ 
 
Objetivo del Evento: 
EXPO TECH 2025 tiene como propósito reunir a líderes de la industria, emprendedores y 
entusiastas de la tecnología para explorar las últimas innovaciones y tendencias del mundo 
digital. Es una plataforma que facilita el acceso a los avances más innovadores de la 
industria tecnológica, ofreciendo una visión completa de las tendencias actuales.  
 
Beneficios y ventajas para los Expositores: 
Al participar como expositor en EXPO TECH, disfrutará de una plataforma excepcional para 
promocionar sus

In [15]:
# Perform a hybrid search using the hybrid_search method
docs = vector_store.hybrid_search(
    query="cuales son las ventajas de participar de la expotech?", k=3
)
print(docs[0].page_content)

EXPOTECH VI Edición, Panamá 2025 
 
Descripción General: 
EXPO TECH 2025 es la feria tecnológica más destacada del año en Panamá, organizada por 
la Cámara de Comercio, Industrias y Agricultura de Panamá (CCIAP). Su Sexta Edición se 
llevará a cabo del 25 al 27 de marzo de 2025 en el Panama Convention Center, ubicado en 
Amador, Ciudad de Panamá. El horario de la expo será de 11:00AM a 7:00PM.  El website 
oficial de la ExpoTech es https://www.panacamara.com/expotech/ 
 
Objetivo del Evento: 
EXPO TECH 2025 tiene como propósito reunir a líderes de la industria, emprendedores y 
entusiastas de la tecnología para explorar las últimas innovaciones y tendencias del mundo 
digital. Es una plataforma que facilita el acceso a los avances más innovadores de la 
industria tecnológica, ofreciendo una visión completa de las tendencias actuales.  
 
Beneficios y ventajas para los Expositores: 
Al participar como expositor en EXPO TECH, disfrutará de una plataforma excepcional para 
promocionar sus

In [49]:
from langchain_ollama import ChatOllama, OllamaLLM
from langchain.tools import tool

llm = ChatOllama(
    model="llama3.2",
    temperature=0,
    # other params...
)

# llm = OllamaLLM(
#     model="llama3.2",
#     temperature=0,
# )


@tool
def informacion_bruno_mottini(nombre_usuario: str):
    """use this tool when user need information about Bruno Mottini
    nombre_usuario: use CACACA"""

    return "bruno mottini un gran gran representante de la inteligencia y abundancia del mundo en el 2025"

tools = [informacion_bruno_mottini]


modelo = llm.bind_tools(tools)

In [50]:
respuesta = modelo.invoke("give me information about Bruno Mottini?")
print(respuesta)

content='' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2024-11-28T14:02:59.5921571Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4831334200, 'load_duration': 22696300, 'prompt_eval_count': 184, 'prompt_eval_duration': 2065000000, 'eval_count': 25, 'eval_duration': 2741000000, 'message': Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='informacion_bruno_mottini', arguments={'nombre_usuario': 'CACACA'}))])} id='run-69c728e9-c874-4967-8ef9-ecccaf808fc3-0' tool_calls=[{'name': 'informacion_bruno_mottini', 'args': {'nombre_usuario': 'CACACA'}, 'id': '8b29b12e-f4d6-43cc-ab16-cec80f54e9aa', 'type': 'tool_call'}] usage_metadata={'input_tokens': 184, 'output_tokens': 25, 'total_tokens': 209}
